In [1]:
import torch as t
import numpy as np
from torch.utils.data import DataLoader
from torch import optim
from torch import nn
from model import *
from torchnet import meter
import tqdm
from config import *
from test import *

In [2]:
def train():
    if Config.use_gpu:
        Config.device = t.device("cuda")
    else:
        Config.device = t.device("cpu")
    device = Config.device
    # 获取数据
    datas = np.load("tang.npz", allow_pickle=True)
    data = datas['data']
    ix2word = datas['ix2word'].item()
    word2ix = datas['word2ix'].item()
    data = t.from_numpy(data)
    dataloader = DataLoader(data,
                            batch_size=Config.batch_size,
                            shuffle=True,
                            num_workers=2)

    # 定义模型
    model = PoetryModel(len(word2ix),
                        embedding_dim=Config.embedding_dim,
                        hidden_dim = Config.hidden_dim)
    Configimizer = optim.Adam(model.parameters(),lr=Config.lr)
    criterion = nn.CrossEntropyLoss()
    if Config.model_path:
        model.load_state_dict(t.load(Config.model_path,map_location='cpu'))
    # 转移到相应计算设备上
    model.to(device)
    loss_meter = meter.AverageValueMeter()
    # 进行训练
    f = open('result.txt','w')
    for epoch in range(Config.epoch):
        loss_meter.reset()
        for li,data_ in tqdm.tqdm(enumerate(dataloader), total=3600, desc=f"round {epoch}/50"):
            #print(data_.shape)
            data_ = data_.long().transpose(1,0).contiguous()
            # 注意这里，也转移到了计算设备上
            data_ = data_.to(device)
            Configimizer.zero_grad()
            # n个句子，前n-1句作为输入，后n-1句作为输出，二者一一对应
            input_,target = data_[:-1,:],data_[1:,:]
            output,_ = model(input_)
            # print("Here",output.shape)
            # 这里为什么view(-1)
            # print(target.shape,target.view(-1).shape)
            loss = criterion(output,target.view(-1))
            loss.backward()
            Configimizer.step()
            loss_meter.add(loss.item())
            # 进行可视化
            if (1+li)%Config.plot_every == 0:
                print("\n训练损失为%s"%(str(loss_meter.mean)))
                f.write("训练损失为%s"%(str(loss_meter.mean)))
                for word in list(u"春江花朝秋月夜"):
                    gen_poetry = ''.join(generate(model,word,ix2word,word2ix))
                    print(gen_poetry)
                    f.write(gen_poetry)
                    f.write("\n\n\n")
                    f.flush()
        t.save(model.state_dict(),'%s_%s.pth'%(Config.model_prefix,epoch))



train()

round 0/50:   6%|███▌                                                             | 199/3600 [16:01<4:32:36,  4.81s/it]


训练损失为1.0140921401977545
春睡夏髭夜，门亭东巷秋。松斋清霁夜，天暖赐灵侯。省隐诸山险，清词万户游。举朝江上寺，看话镜中头。出户延金石，回光媚玉楼。金泥容度顾，桂魄醉骖摇。逸兴初凭槛，孤阳已带舟。竹开山自重，坛设月难留。幽籁传琼液，庭梧洗桂衣。海衣红粉赋，宾旧几淹留。水与秦尘变，天长岁月流。那堪远山色，万里到吴州。
江县传家府，云帆际五湖。应依钓梁郭，献赋在墙东。位带陶侯落，文慙汉主中。诏然纷列列，勋肃冠轩雄。既郡鼙惊府，推题岁月丰。受兹良未遂，寂寞道仍穷。瓶冷光全直，琴杯骨素终。令恩羁近近，仆政在凉风。公道时来代，中宵思有风。望遥因暇景，暂別谢玄蒙。浪隔乘轺鹤，歌高接䌽风。四时恭宴往，三载奉承同。远壑遥通楚，浮光下又东。分金更汲引，似月使丝红。时忆逢诗客，荷携钓钓翁。苍茫遥望见，湓浦迥闻风。月下离杯晚，天涯
花馆见汝农人，在中多之空山。朝为水长路，水遶山郭间。杂竹夹枯松，苍苍倚冈鬘。半壁无昼头，拔冈尽千里。峨眉青苍，万顷沈潇珠。越女解弦管，京凉满粧翰。流年自有时，反物有余安。万象合海转，一峰间水坳。阴阳精爱栗，大泽游丹霞。攒碧冒夕华，绿蔓生小窗。悬旭清鉴中，理地看琼花。始知繁华好，可以游莲花。芳草有青青，长条满朝华。青天欲可升，黄鸟不自裁。愿持百万岁，目送春风多。胡为不计时，千花落颜一。
朝光入洛阳，羣女多繁枝。四子无禁容，姑眉弄春姿。二十四十人，将死无情期。青桂仍载落，苍然永无期。但看蓬莱池，东家车马奇。曾闻五龙窟，去往无纤丝。仙人眷中国，玉为鸞凤姿。万乘飞羽翼，双舞罗琼诗。同为玉皇游，君游上楼诗。含情舞珠仪，疑我清昼期。昨从春色生，连回春风迟。莫辞绿罗远，又觉六愁衰。自此长叹息，不知何所之。
秋天月，落月孤舟送客。江人愁坐来无容，暮帆离別但为言。潇湘水阔湘水清，舳门寒烟细烟轻。牛羊劝我蒲杂指，啄虫惊鸣不相触。侧身入林下破冰，孤舟欲耕白似牛。兰舟旋腾百里疾，帆影却到行处秋。坐来灯下话明主，瞥忆乡中白日归。扬鞭出塞横萧萧，中家战骑无轻休。青门春草侵尔时，四门万卷皆是非。到口军前如擣马，当时手有素衣归。去来计役为移社，他日为官尽举围。试上大罗与天別，两度将军各奔凑。何当遂饮铜公游，还自东归值
月下天半树，水边寒景薄。行看闾里翁，偶向南津路。相逢新岁月，谁家少邻里？夜日空暮云，离眉遥隔水。诗倾衣冠文，石上酒满杯。洛阳女稚马，今为鹧鸪鸣。池水化为水

round 0/50:   6%|███▌                                                             | 200/3600 [16:10<5:50:58,  6.19s/it]

夜梦神妃琴，山头凫兕电。手堕金玉经，岂比神仙窟。有时把一枝，水面两盘似。崩剥泉石上，立地多薪闹。自压諠太脚，始知施饭大。五音定不下，众肠而倚裂。千岁遭轮一，尽死最横脊。呜呜大角角，胆乱掉罗裂。天怒太陨鼻，恰得精灵理。意气於其间，无心为道鬼。洗心出云门，一真不肯拔。果得一点甲，呼不神失止。若穿笼处足，如是逢太至。早岸何曾连，短船无一力。圆复三月精，中夜有六月。有如丰。


round 0/50:  11%|███████▏                                                         | 399/3600 [33:10<4:18:59,  4.85s/it]

...


训练损失为1.5363887048851368
春江春气生，茅屋对庭前。鸳鸯卷卷衣，翡翠依莲碧。阶前露华动，径上帷阁绝。滴滴残月中，凉风飒飘飖。冷烟笼户牖，疎叶笼江叶。过水连古林，栖禽出彭泽。豁然声寂寂，何暇知心耳。客为远山云，想入洞庭目。因思长林下，独宿高云广。山木含故音，灵源去何极。石门积霞外，千里凝积霭。长驱万里外，日夕随行织。
江上景沈沈沈沈，雪崩左固雄豪。开元卢儿庙下，走马鞭羸林。值我交役不得，况闻下山川。孟陵堰火魂，直上白日沉。下视市山溃，次第千仞林。水流不敢恃，拙俗谁复侵。君看天竺心，半落千仞岑。我今与我怀，各各营一间。我行岂云远，私爱不复间。君看云中松，结根常氤氲。寄语松柏孙，有献甘泉山。何必徇羽翰，翻为古风吟。因循战粟间，未免身侵侯。顷日谢公起，丘园一何钦。
花屏阁上茶，真字机与善。既是奇烈人，不及寻常全。试试杂芳酒，却求一大牙。内日无他骂，愚缘摄共安。内堂为择醪，灯灰诤有盘。团团有常珠，截地不能看。薪毒毒而耗，空肠难可谩。夙日即有因，贪恶不死走。不假铅与汞，铅金火生炭。服者错增谬，骂家恶即失。直使无他人，对面便有有。不及理离事，却遣凡死理。不断有时起，永永入佛脉。
朝阳四百年，十亩六株柳。明月照南湖，长林秋夜半。凄凄开横户，明旭澄复澈。是时接明月，长河如落月。照山上天半，夹地明明景。蹴视羣木莲，森森积微岭。樵歌何所惬，楚竹堪怅望。瑶水醉将飞，华轩会衰醁。南迁梦沦袂，漂泊故乡泪。丝网随猜肤，炎炎或麤蹇。层峦盘石干，傍曜资蛙腻。渡海洗颓原，连函阻羣牧。兹路何在兹，幽期不可数。寄言沧浪客，一顾沦一夕。愿言谢风便，自此谢安阙。
秋天高高秋水清，乌楼北风吹我声。觉来潜忆衡山宿，始应一声钟磬断。君不见高丘有此心，独有一人听寒雁。续闻啼，惊寄目。风嫋碎，波不动，风飕萧萧风雨起。思师上策空余言，可怜行兮不与闻。情余反愁心莫测，我亦不知其有信。但知天语思翛然，羲炳琅玕岂知足。行云汉上两青青，一雨一声秋水清。
月谷婆子真，幽姿自相望。既无纤尘力，盥尔在我傍。忽然无处宽，此地得以愆。我今有我时，各各济暮途。我居悄无言，再食如有虞。惟闻不相顾，千载为文颜。奈何从此时，草木自成刳。顾我非苦怀，默默而永矣。虽无佳人心，竟岁多蹇惕。愿从萧曹吏，跼沐在时赐。俄想庐谿宽，偃仰俯幽匮。念尔遂时喜，琴谈为君过。叹息形外间，闲观遂兹朗。幸承仙公殿，何用居品迹。


round 0/50:  61%|█████████████████████████████████████▉                        | 2200/3600 [3:00:05<2:26:18,  6.27s/it]

夜梦乌童妇夜来，如何打下复连宵。少年贤道多结束，好趂诸侯论异诸。学任清溪游海内，身随楚水逐横调。邛城万里云如水，出入王孙日几回。还为桑田卑未尽，况应蓟北对豺堆。须期日夕因风追，故人山头云海涛。铜驼万丈」无敌，不似今朝未得去。借问秋风何处栖，黄昏罢沥井中飞。校得江连杜陵客，不知今日是他旗。高山大抵无行路，一段平沙愁寂寞。莫使流年归故乡，一道雍门看一宿。


round 0/50:  65%|████████████████████████████████████████                      | 2328/3600 [3:10:29<1:44:05,  4.91s/it]


KeyboardInterrupt: 